In [1]:
from pyspark.sql import *
from pyspark import SparkContext, SparkConf
import json
import numpy as np
import requests
from pyspark.sql import dataframe
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder \
    .master("local") \
    .appName("HistData") \
    .config("spark.debug.maxToStringFields", "1000") \
    .getOrCreate()


In [4]:
def getResourceDict (json):
    # по листу метрики из json данных возвращает словарь процессов
    list_data = json['data']['result'] # список словарей по процессам
    # предобработка
#     for i in list_data:
#         for j in range(len(i["values"])):
#             i["values"][j][1] = int(i["values"][j][1])
    # values_list = list(map(lambda x: x["values"], list_data))
    resourceTypeDict = dict(map(lambda x: (x["metric"]["resource_type"], x["values"]), list_data))
    # iterObj = iter(resourceTypeDict)
    # print(list(iterObj))
    return resourceTypeDict

In [5]:
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()

with open('C:/Users/Lenovo/Desktop/repoMon/MonitoringSystem/query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print(len(column_list))
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

df = None
for el in column_list[:5]:
    df_schema = StructType([StructField("Time", FloatType(), True), StructField(el, DoubleType(), True)])
    if df is None:
        df = spark.createDataFrame(resourceTypeDict[el])
        df = df.toDF("Time", el)
    else:
        df_right = spark.createDataFrame(resourceTypeDict[el]).toDF("Time", el)
        # df_right[el] = df_right[el].withColumn(el, df_right[el].cast('int').alias(el))
        # df_right[el] = df_right[el].cast(FloatType())
        df = df.join(df_right, "Time", "right")

print(f"#### Columns: {len(df.columns)} #### {df.count()}")
df.printSchema()

3324
#### Columns: 6 #### 37
root
 |-- Time: double (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_0: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_1: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_10: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_11: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_2: string (nullable = true)



In [11]:
df.show(5)

+----------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+----------------------------------+
|            Time|AGMService.exe_5208_1604522690.0_0|AGMService.exe_5208_1604522690.0_1|AGMService.exe_5208_1604522690.0_10|AGMService.exe_5208_1604522690.0_11|AGMService.exe_5208_1604522690.0_2|
+----------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+----------------------------------+
|1.604762586663E9|                           7327744|                           3756032|                            3911680|                            3756032|                              5587|
|1.604762241663E9|                           7471104|                           3756032|                            3911680|                            3756032|                              5587|
|1.604762361663E9|  

In [16]:
import pandas as pd
from IPython.display import display

pd.options.display.max_columns = None

pandasDF = df.toPandas()
display(pandasDF.head())


,Time,AGMService.exe_5208_1604522690.0_0,AGMService.exe_5208_1604522690.0_1,AGMService.exe_5208_1604522690.0_10,AGMService.exe_5208_1604522690.0_11,AGMService.exe_5208_1604522690.0_2
0,1.604763e+09,7327744,3756032,3911680,3756032,5587
1,1.604762e+09,7471104,3756032,3911680,3756032,5587
2,1.604762e+09,7446528,3756032,3911680,3756032,5587
3,1.604762e+09,7446528,3756032,3911680,3756032,5587
4,1.604763e+09,7438336,3756032,3911680,3756032,5587


Ячейка ниже - копия логики создания объединенного датафрейма но на пандасе (1001 столбец генерит за несколько секунд)
Для перевода df.pandas в Dfpyspark расскоментить отмеченную строку (есть баг при конвертации большого числа столбцов (>100), скорей всего связанный с появлением Нанов для которых нужно указать корректную схему df пайспарка)

In [6]:
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()
import pandas as pd
with open('C:/Users/Lenovo/Desktop/repoMon/MonitoringSystem/query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print(len(column_list))
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

pdf = None
for el in column_list[:50]: #
#     df_schema = StructType([StructField("Time", FloatType(), True), StructField(el, DoubleType(), True)])
    if pdf is None:
        pdf = pd.DataFrame(resourceTypeDict[el], columns=["Time", el])
#         df = df.toDF("Time", el)
    else:
        pdf_right = pd.DataFrame(resourceTypeDict[el], columns=["Time", el])
#         .toDF("Time", el)
        # df_right[el] = df_right[el].withColumn(el, df_right[el].cast('int').alias(el))
        # df_right[el] = df_right[el].cast(FloatType())
        pdf = pdf.join(pdf_right.set_index("Time"), on ="Time")

# df_spark = spark.createDataFrame(pdf)
print(f"#### Columns: {len(pdf.columns)} #### ")
pdf
# df.show()

3324
#### Columns: 51 #### 


,Time,AGMService.exe_5208_1604522690.0_0,AGMService.exe_5208_1604522690.0_1,AGMService.exe_5208_1604522690.0_10,AGMService.exe_5208_1604522690.0_11,AGMService.exe_5208_1604522690.0_2,AGMService.exe_5208_1604522690.0_3,AGMService.exe_5208_1604522690.0_4,AGMService.exe_5208_1604522690.0_5,AGMService.exe_5208_1604522690.0_6,...,AcroRd32.exe_20736_1604753700.0_2,AcroRd32.exe_20736_1604753700.0_3,AcroRd32.exe_20736_1604753700.0_4,AcroRd32.exe_20736_1604753700.0_5,AcroRd32.exe_20736_1604753700.0_6,AcroRd32.exe_20736_1604753700.0_7,AcroRd32.exe_20736_1604753700.0_8,AcroRd32.exe_20736_1604753700.0_9,AdminService.exe_4788_1604522690.0_0,AdminService.exe_4788_1604522690.0_1
0,1.604762e+09,7471104,3756032,3911680,3756032,5587,11341824,7471104,138968,138968,...,19829,38535168,20430848,271904,249216,29104,26096,13590528,5124096,17002496
1,1.604762e+09,7471104,3756032,3911680,3756032,5587,11341824,7471104,138968,138968,...,19829,38535168,20430848,271904,249216,29104,26096,13590528,5124096,17002496
2,1.604762e+09,7467008,3756032,3911680,3756032,5587,11341824,7467008,138968,138968,...,19829,38535168,20418560,271904,249216,29104,26096,13590528,5124096,17002496
3,1.604762e+09,7467008,3756032,3911680,3756032,5587,11341824,7467008,138968,138968,...,19829,38535168,20418560,271904,249216,29104,26096,13590528,5124096,17002496
4,1.604762e+09,7454720,3756032,3911680,3756032,5587,11341824,7454720,138968,138968,...,19829,38535168,20402176,271904,249216,29104,26096,13590528,5124096,17002496
5,1.604762e+09,7454720,3756032,3911680,3756032,5587,11341824,7454720,138968,138968,...,19829,38535168,20402176,271904,249216,29104,26096,13590528,5124096,17002496
6,1.604762e+09,7454720,3756032,3911680,3756032,5587,11341824,7454720,138968,138968,...,19829,38535168,20402176,271904,249216,29104,26096,13590528,5111808,17002496
7,1.604762e+09,7446528,3756032,3911680,3756032,5587,11341824,7446528,138968,138968,...,19829,38535168,20377600,271904,249216,29104,26096,13590528,5111808,17002496
8,1.604762e+09,7446528,3756032,3911680,3756032,5587,11341824,7446528,138968,138968,...,19829,38535168,20377600,271904,249216,29104,26096,13590528,5111808,17002496
9,1.604762e+09,7446528,3756032,3911680,3756032,5587,11341824,7446528,138968,138968,...,19829,38535168,20377600,271904,249216,29104,26096,13590528,5111808,17002496


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

